In [1]:
import pynq

ol = pynq.Overlay('test_dma_stream.bit', download=True)

In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=1, width=80, depth=3)
#pp.pprint(ol.ip_dict['axi_dma_0'])
#pp.pprint(ol.ip_dict['count_stream_test_0'])

In [3]:
from pynq import allocate # for DMA interactions

def chan_status(chan):
    if chan is None:
        return ''
    
    r = chan.running
    i = chan.idle
    e = chan.error

    return f'Run:{r} Idle:{i} Err:{e}'

def dump_regs(dma, offset):
    info = ''
    cr = dma.mmio.read(offset + 0x0)
    sr = dma.mmio.read(offset + 0x4)
    lo = dma.mmio.read(offset + 0x18)
    hi = dma.mmio.read(offset + 0x1c)
    sz = dma.mmio.read(offset + 0x28)
    info += f'CR:   0x{cr:08x}\n'
    info += f'SR:   0x{sr:08x}\n'
    info += f'addr: 0x{hi:08x}{lo:08x}\n'
    info += f'len:  0x{sz:08x}\n'
    return info

def dump_stream_regs(count_stream):
    info = ''
    lo    = count_stream.read(0x0)
    hi    = count_stream.read(0x4)
    info += f'StreamCfg: {lo} to {hi}\n'
    debug = count_stream.read(0xc)
    info += f'TVALID: {(debug & (1 << 0)) >> 0} '
    info += f'TLAST: {(debug & (1 << 1)) >> 1} '
    info += f'TREADY: {(debug & (1 << 2)) >> 2}\n'
    info += f'counter_start: {(debug & (1 << 4)) >> 4}\n'
    info += f'read_pointer: {debug >> 16} '
    info += f'tx_en: {(debug & (1 << 8)) >> 8}\n'
    return info


def debug_dma(dma):
    if dma.sendchannel is not None:
        print('Send Channel', chan_status(dma.sendchannel))
        print(dump_regs(dma, 0x0))
    if dma.recvchannel is not None:
        print('Recv Channel', chan_status(dma.recvchannel))
        print(dump_regs(dma, 0x30))

def debug_stream(count_stream):
    print(dump_stream_regs(count_stream))
    
def debug_both(dma, count_stream):
    debug_dma(dma)
    debug_stream(count_stream)
    

In [4]:
count = 5
start = 1

debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)
dma_result = allocate(shape=(count,), dtype='u8')
print(dma_result)
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)
ol.axi_dma_0.recvchannel.transfer(dma_result, nbytes=count*8)
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)
ol.count_stream_test_0.write(0x0, start)         # counter_lo
ol.count_stream_test_0.write(0x4, start+count-1) # counter_hi
ol.count_stream_test_0.write(0x8, 1)             # counter start
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)
ol.axi_dma_0.recvchannel.wait()
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)
print(dma_result)

Recv Channel Run:True Idle:False Err:False
CR:   0x00011003
SR:   0x00000000
addr: 0x0000000000000000
len:  0x00000000

StreamCfg: 0 to 0
TVALID: 0 TLAST: 1 TREADY: 1
counter_start: 0
read_pointer: 65535 tx_en: 0

[0 0 0 0 0]
Recv Channel Run:True Idle:False Err:False
CR:   0x00011003
SR:   0x00000000
addr: 0x0000000000000000
len:  0x00000000

StreamCfg: 0 to 0
TVALID: 0 TLAST: 1 TREADY: 1
counter_start: 0
read_pointer: 65535 tx_en: 0

Recv Channel Run:True Idle:False Err:False
CR:   0x00011003
SR:   0x00000000
addr: 0x0000000016849000
len:  0x00000028

StreamCfg: 0 to 0
TVALID: 0 TLAST: 1 TREADY: 1
counter_start: 0
read_pointer: 65535 tx_en: 0

Recv Channel Run:True Idle:True Err:False
CR:   0x00011003
SR:   0x00001002
addr: 0x0000000016849000
len:  0x00000028

StreamCfg: 1 to 5
TVALID: 0 TLAST: 0 TREADY: 1
counter_start: 0
read_pointer: 5 tx_en: 0

Recv Channel Run:True Idle:True Err:False
CR:   0x00011003
SR:   0x00001002
addr: 0x0000000016849000
len:  0x00000028

StreamCfg: 1 to 5


In [5]:
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)

Recv Channel Run:True Idle:True Err:False
CR:   0x00011003
SR:   0x00001002
addr: 0x0000000016849000
len:  0x00000028

StreamCfg: 1 to 5
TVALID: 0 TLAST: 0 TREADY: 1
counter_start: 0
read_pointer: 5 tx_en: 0



In [6]:
start = 134
end = 650

count = end - start + 1

result = allocate(shape=(count,), dtype='u8')
ol.axi_dma_0.recvchannel.transfer(result, nbytes=count*8)
ol.count_stream_test_0.write(0x0, start)
ol.count_stream_test_0.write(0x4, end)
ol.count_stream_test_0.write(0x8, 1)
ol.axi_dma_0.recvchannel.wait()
print(result)

[134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151
 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169
 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187
 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205
 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223
 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241
 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259
 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277
 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295
 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313
 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331
 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349
 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367
 368 369 370 371 372 373 374 375 376 377 378 379 38

### Notes

Any size seems to work correctly, now. But I only get half of the data? So weird. Is there some kind of strobing/throttling?

Might wanna testbench count_stream_test_0 as well.

### TODO

* Expand DishSoap class to actually wrap this behavior out
* ????
* start working on the other side of the network sim again

In [7]:
from dishsoap import CountStreamTest, TestDmaStreamOverlay
ol = TestDmaStreamOverlay()

In [8]:
ol.debug_both()

Recv Channel Run:True Idle:False Err:False
CR:   0x00011003
SR:   0x00000000
addr: 0x0000000000000000
len:  0x00000000

StreamCfg: 0 to 0
TVALID: 0 TLAST: 1 TREADY: 1
counter_start: 0
read_pointer: 65535 tx_en: 0

